# **CONTENT**
1. Loading Packages.
2. API connection to spotify.
3. Data transformation from Json to dataset. (Content of artists, songs and genre).
4. Creation of bucket in GCP.
5. Import csv into Big-Query.
6. Selecting data of created datasets.
7. Automation.

# **Installing Packages**

In [ ]:
# Installing Packages
!pip install spotify  -q
!pip install spotipy  -q
!pip install gdown    -q
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import spotify
import spotipy
from datetime import date
import os
import spotify
import spotipy
from datetime import date

# EXTRACTION. API CONNECTION TO SPOTIFY + CONVERTING JSON TO DATAFRAME
---



In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="74d4bbd27a8e4a2fb374a14b6ca41ca7",
                                                           client_secret="80eeca8fdc0d404480c79d0a4c40eca4"))


In [ ]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
track_names = []
artist_uris = []
artists_info = []
artist_names = []
artist_pops = []
artists_genres = []
albums = []
track_pops= []
explicits = []
relased_dates = []
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    #Explicit name
    explicit=track["track"]['explicit']
    explicits.append(explicit)  
    #Track name
    track_name = track["track"]["name"]
    track_names.append(track_name)    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_uris.append(artist_uri)
    artist_info = sp.artist(artist_uri)
    artists_info.append(artist_info)
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_names.append(artist_name)
    artist_pop = artist_info["popularity"]
    artist_pops.append(artist_pop)
    artist_genres = artist_info["genres"]
    artists_genres.append(artist_genres)
    #Album
    album = track["track"]["album"]["name"]
    albums.append(album)
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    track_pops.append(track_pop)
  
    #Release date
    release_date = track['track']['album']['release_date']
    relased_dates.append(release_date)     

In [ ]:
explicit=[int(i) for i in explicits]
import string as str
import pandas as pd
Datos_canciones=pd.DataFrame(
    sp.audio_features(track_uris),
    columns=[
        "danceability",
        "energy",
        "key",
        "loudness",
        "mode",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "tempo",
        "type",
        "id",
        "uri",
        "track_href",
        "analysis_url",
        "duration_ms",
        "time_signature",
        "explicits",
        "relased_dates"
    ],
)
Datos_canciones["popularity"]=track_pops
Datos_canciones["Album"] =albums 
Datos_canciones["Fecha_Extraccion"] = date.today() 
Datos_canciones["artist_name"]= artist_names
Datos_canciones["track_name"]= track_names 
Datos_canciones["Artist_uris"]=artist_uris
Datos_canciones["relased_dates"]=relased_dates
Datos_canciones["explicits"]=explicits
Datos_canciones["Artist_uris"]=artist_uris
Datos_canciones['Artist_uris'] =Datos_canciones['Artist_uris'].str.replace('spotify:artist:', '')
Datos_canciones['uri'] =Datos_canciones['uri'].str.replace('spotify:track:', '')

Datos_artista=pd.DataFrame(
    columns=[
        "artists",
        "artist_name",
        "track_name",
        "Artist_uris",
        "Artist_Info",
        "Artist_pop"
    ],
)
Datos_artista["artists"]= artist_names
Datos_artista["artist_name"]=artist_names
Datos_artista["track_name"]= track_names 
Datos_artista["Artist_uris"]=artist_uris
Datos_artista['Artist_uris'] = Datos_artista['Artist_uris'].str.replace('spotify:artist:', '')
#Datos_artista["Artist_Info"]= artists_info 
Datos_artista["Artist_pop"] = artist_pops
Datos_artista["Fecha_Extraccion"] = date.today() 

Datos_genero=pd.DataFrame(
    columns=[
        "Artist_uris",
        "Artist Genre"
            ],
)

Datos_genero["Artist_uris"]=artist_uris
Datos_genero['Artist_uris'] =Datos_genero['Artist_uris'].str.replace('spotify:artist:', '')

Datos_genero["Artist Genre"] = artists_genres 
Datos_genero['Main Genre']=Datos_genero['Artist Genre'].str[0]  


Datos_artista=Datos_artista.drop_duplicates()
Datos_canciones=Datos_canciones.drop_duplicates()
Datos_genero=Datos_genero.drop(['Artist Genre'], axis=1)
Datos_genero=Datos_genero.drop_duplicates()

# **TRANSFORMING THE DATASET. PREPARING TABLES.**
---

In [ ]:
Datos_artista.to_csv("Datos_artista.csv", index=False)
Datos_canciones.to_csv("Datos_canciones.csv", index=False)
Datos_genero.to_csv("Datos_genero.csv", index = False)

# **IMPORTING TEXT INTO BUCKET GCP**

In [ ]:
from google.cloud import storage
import pandas as pd
import json


key_path = "/content/plasma-yeti-380204-c14f404dabe9.json"
# Create a client
client = storage.Client.from_service_account_json(key_path)

# Set the name of the bucket to which you want to upload the file
bucket_name = "prueba1111"
# Get a reference to the bucket
bucket = client.bucket(bucket_name)

# Get the bucket containing the file you want to download
bucket = client.bucket('prueba1111')

# Get a blob (file) from the bucket
blob = bucket.blob('plasma-yeti-380204-c14f404dabe9.json')
data = blob.download_as_string()
# Save the JSON data to a local file
with open('my_credentials.json', 'wb') as f:
    f.write(data)
# Create credentials from the downloaded file
creds = storage.Client.from_service_account_json('my_credentials.json')
# Path to your service account key file
key_path = '/content/my_credentials.json'
# Create a client
client = storage.Client.from_service_account_json(key_path)

# Set the name of the bucket to which you want to upload the file
bucket_name = "prueba1111"

# Set the name of the file you want to upload to the bucket
file_name = "Datos_artista.csv"
file_name1 = "Datos_canciones.csv"
file_name2 = "Datos_genero.csv"
# Set the path of the file on your local machine
file_path = "/content/Datos_artista.csv"
file_path1= "/content/Datos_canciones.csv"
file_path2= "/content/Datos_genero.csv"
# Get a reference to the bucket
bucket = client.bucket(bucket_name)

# Create a blob object representing the file to be uploaded
blob = bucket.blob(file_name)
blob1 = bucket.blob(file_name1)
blob2 = bucket.blob(file_name2)

# Upload the file to the bucket
blob.upload_from_filename(file_path)
blob.upload_from_filename(file_path1)
blob.upload_from_filename(file_path2)

# **LOAD DATASET INTO GCP BUCKET.**

Creation of Tables and Insertion of dataset.*italicized text*

Load bucket GCP into GCP Big Query



In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "plasma-yeti-380204-c14f404dabe9.json"
from google.cloud import bigquery
client1 = bigquery.Client(project="270255849444")

# Crear tabla.
QUERY = ('LOAD DATA OVERWRITE spotify1.artista FROM FILES (format = "CSV", uris = ["gs://prueba1111/Datos_artista.csv"])')
query_job = client1.query(QUERY)  # API request
results=query_job.to_dataframe()
results;


QUERY = ('LOAD DATA OVERWRITE spotify1.canciones1 FROM FILES (format = "CSV", uris = ["gs://prueba1111/Datos_canciones.csv"])')
query_job = client1.query(QUERY)  # API request
results=query_job.to_dataframe()
results;


QUERY = ('LOAD DATA OVERWRITE spotify1.genero FROM FILES (format = "CSV", uris = ["gs://prueba1111/Datos_genero.csv"])')
query_job = client1.query(QUERY)  # API request
results=query_job.to_dataframe()
results;

# **READING QUERIES + JOINS**



In [ ]:
# Listar Query
QUERY = (
    'select * from spotify1.prueba2 limit 5')
query_job = client1.query(QUERY)  # API request
results=query_job.to_dataframe()
results

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,analysis_url,duration_ms,time_signature,popularity,Album,Fecha_Extraccion,artist_name,track_name,Artist_uris,relased_dates
0,0.464,0.417,0,-9.345,0,0.0256,0.13600,0.022000,0.0974,0.479,...,https://api.spotify.com/v1/audio-analysis/5XeF...,183956,4,90,AM,2023-03-19,Arctic Monkeys,I Wanna Be Yours,spotify:artist:7Ln80lUS6He07XvHI8qqHH,2013-09-09
1,0.575,0.500,1,-7.594,0,0.0707,0.22700,0.000000,0.3350,0.506,...,https://api.spotify.com/v1/audio-analysis/4W4f...,232857,4,61,Starboy (Deluxe),2023-03-19,The Weeknd,Die For You (with Ariana Grande) - Remix,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,2023-03-14
2,0.715,0.620,1,-6.005,0,0.0484,0.41700,0.000000,0.0822,0.172,...,https://api.spotify.com/v1/audio-analysis/2dHH...,221520,4,95,HEROES & VILLAINS,2023-03-19,Metro Boomin,Creepin' (with The Weeknd & 21 Savage),spotify:artist:0iEtIxbK0KxaSlF7G42ZOp,2022-12-02
3,0.586,0.525,1,-7.163,0,0.0615,0.11100,0.000000,0.1340,0.508,...,https://api.spotify.com/v1/audio-analysis/2LBq...,260253,4,88,Starboy,2023-03-19,The Weeknd,Die For You,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,2016-11-24
4,0.901,0.589,1,-6.713,0,0.2890,0.00302,0.000237,0.0992,0.399,...,https://api.spotify.com/v1/audio-analysis/1zsP...,196600,4,89,OzuTochi,2023-03-19,Ozuna,Hey Mor,spotify:artist:1i8SpTcr7yvPOmcqrbnVXY,2022-10-07


# AUTOMATION.

https://cloud.google.com/vertex-ai/docs/workbench/managed/schedule-managed-notebooks-run-quickstart?hl=es-419

In [ ]:
#https://console.cloud.google.com/vertex-ai/workbench/managed?hl=es-419&project=plasma-yeti-380204
#https://6bf7b36e1fb668c2-dot-us-central1.notebooks.googleusercontent.com/lab/tree/2_API%2BETL_Spotify.ipynb
#https://console.cloud.google.com/vertex-ai/workbench/locations/us-west1/managed/managed-notebook-1679217573?project=plasma-yeti-380204
